In [1]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score




Image ID                                           1507518827
Data        [[[  7   3  13]\n  [ 15  12  40]\n  [ 21   7  ...
Name: 0, dtype: object
